# Subgraphs

Create modular, reusable graph components using subgraphs.

## Learning Objectives

This notebook will teach you advanced LangGraph concepts through practical examples.

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
print("✓ Setup complete!")

## Coming Soon

This notebook is under development. Check back soon for comprehensive examples and exercises!